In [134]:
import pandas as pd


In [135]:
movies = pd.read_csv('movies.csv')
links = pd.read_csv('links.csv')


In [136]:
movies = movies.merge(links, on='movieId')


In [137]:
movies.head(1)


,movieId,title,genres,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0


In [144]:
movies1 = movies.iloc[:5000, :]
movies2=movies.iloc[20000:25000,:].reset_index()
movies3=movies.iloc[40000:45000,:].reset_index()
# movie=pd.concat(movies1,movies2)

,index,movieId,title,genres,imdbId,tmdbId
0,20000,103685,"Field in England, A (2013)",Drama|Fantasy|Thriller|War,2375574,179111.0
1,20001,103688,"Conjuring, The (2013)",Horror|Thriller,1457767,138843.0
2,20002,103690,Everything (2004),Drama|Mystery,430065,28698.0
3,20003,103692,Strawberry Fields (2012),Drama,1637656,137752.0
4,20004,103695,"Starfighters, The (1964)",Drama,58615,31122.0
...,...,...,...,...,...,...
4995,24995,122760,My 5 Wives (2000),Comedy,216930,38132.0
4996,24996,122762,My Brother the Pig (1999),Children|Comedy|Fantasy|Sci-Fi,165396,73257.0
4997,24997,122764,Napoleon (1995),Adventure|Children,113932,44381.0
4998,24998,122766,No Tell Motel (2013),Horror,2049511,121071.0


In [142]:
movie.head(11)
# movie.tail(11)

,movieId,title,genres,imdbId,tmdbId
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


In [7]:
def title(value):
    value = value.replace(" ", "")
    c = value[-2:-6:-1]
    if(c.isdigit()):
        return int(c[::-1])
    else:
        return None


movies['year'] = movies['title'].apply(title)


In [8]:
movies.dtypes


movieId      int64
title       object
genres      object
imdbId       int64
tmdbId     float64
year       float64
dtype: object

In [9]:
movies['year'].value_counts()


2009.0    945
2008.0    879
2011.0    854
2012.0    838
2010.0    821
         ... 
1894.0      1
1896.0      1
1903.0      1
1899.0      1
1900.0      1
Name: year, Length: 115, dtype: int64

In [10]:
movies.isnull().sum()


movieId      0
title        0
genres       0
imdbId       0
tmdbId     100
year         2
dtype: int64

In [11]:
movies.dropna(inplace=True)


In [12]:
movies.isnull().sum()


movieId    0
title      0
genres     0
imdbId     0
tmdbId     0
year       0
dtype: int64

In [13]:
movies['year'] = movies['year'].astype('int64')


In [14]:
movies['year'].min()


1894

In [15]:
movies['year'].max()


2015

In [16]:
movies['tmdbId'] = movies['tmdbId'].astype('int64')


In [17]:
movies.dtypes


movieId     int64
title      object
genres     object
imdbId      int64
tmdbId      int64
year        int64
dtype: object

In [18]:
def genres(i):
    l = []
    i = i.replace('|', ',')
    l.extend(i.split(','))
    return l


movies['genres'] = movies['genres'].apply(genres)


In [19]:
movies['genres'] = movies['genres'].apply(lambda x: " ".join(x))


In [20]:
movies['genres']


0        Adventure Animation Children Comedy Fantasy
1                         Adventure Children Fantasy
2                                     Comedy Romance
3                               Comedy Drama Romance
4                                             Comedy
                            ...                     
24995                                         Comedy
24996                 Children Comedy Fantasy Sci-Fi
24997                             Adventure Children
24998                                         Horror
24999                                Horror Thriller
Name: genres, Length: 24898, dtype: object

In [21]:
from sklearn.feature_extraction.text import CountVectorizer


In [22]:
cv = CountVectorizer()


In [23]:
vectors = cv.fit_transform(movies['genres']).toarray()


In [24]:
cv.get_feature_names()


C:\Users\Lenovo\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['action',
 'adventure',
 'animation',
 'children',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'fantasy',
 'fi',
 'film',
 'genres',
 'horror',
 'imax',
 'listed',
 'musical',
 'mystery',
 'no',
 'noir',
 'romance',
 'sci',
 'thriller',
 'war',
 'western']

In [25]:
from sklearn.metrics.pairwise import cosine_similarity
similar_movies = cosine_similarity(vectors)


In [26]:
def title2(value):
    aa = value.split()
    bb = aa[:len(aa)-1]
    return " ".join(bb)


movies['title'] = movies['title'].apply(title2)


In [27]:
movies.loc[10, 'title'] = 'The American President'


In [28]:
movies['oldtitle'] = movies['title']


In [29]:
movies['oldtitle'] = movies['oldtitle'].apply(lambda x: x.lower())
movies['oldtitle'] = movies['oldtitle'].apply(lambda x: x.replace(" ", ""))


In [124]:
def recommand(movie):
    movie = movie.lower().replace(" ", "")
    if movie not in movies['oldtitle'].values:
        return "No Movie found,please try again"
    else:
        movie_index = movies[movies['oldtitle'] == movie].index[0]
        distances = enumerate(list(similar_movies[movie_index]))
        j = sorted(distances, reverse=True, key=lambda x: x[1])
        # return j
        for i in range(0,20000):
            # print(i)
            if(j[i][0]==movie_index):
                j=j[i:]
        movie_rec = j[:10]
        l = []
        for i in movie_rec:
            index, distance = i
            l.append(movies.loc[index, 'title'])
    return l


a = input("enter the movie name: ")
b = recommand(a)
b

['Love',
 'Wrestler, The',
 'Double Dynamite',
 'Revolutionary Road',
 'Surfer, Dude',
 'Defiance',
 'Poultrygeist: Night of the Chicken Dead',
 'Have Rocket, Will Travel',
 'Wendy and Lucy',
 'Open Season 2']

In [96]:
correct = " ".join(b).lower()
correct


"nixon othello dangerous minds cry, the beloved country restoration georgia home for the holidays mr. holland's opus two bits the journey of august king margaret's museum boys of st. vincent, the star maker, the (uomo delle stelle, l') silences of the palace, the (saimt el qusur) angela neon bible, the shadows (cienie) gospa basketball diaries, the awfully big adventure, an beauty of the day (belle de jour) feast of july kids nadja showgirls stars fell on henrietta, the white man's burden browning version, the burnt by the sun (utomlyonnye solntsem) cure, the exotica jefferson in paris little women ladybird ladybird enfer, l' my crazy life (mi vida loca) losing isaiah miracle on 34th street my family nell panther pushing hands (tui shou) priest quiz show three colors: red (trois couleurs: rouge) three colors: blue (trois couleurs: bleu) red firecracker, green firecracker (pao da shuang deng) strawberry and chocolate (fresa y chocolate) to live (huozhe) tom & viv"

In [97]:
a = a.lower()
e = len(a)
d = correct.find(a)
correct[d: d+e]

'love'

In [31]:
movies.head(10)


,movieId,title,genres,imdbId,tmdbId,year,oldtitle
0,1,Toy Story,Adventure Animation Children Comedy Fantasy,114709,862,1995,toystory
1,2,Jumanji,Adventure Children Fantasy,113497,8844,1995,jumanji
2,3,Grumpier Old Men,Comedy Romance,113228,15602,1995,grumpieroldmen
3,4,Waiting to Exhale,Comedy Drama Romance,114885,31357,1995,waitingtoexhale
4,5,Father of the Bride Part II,Comedy,113041,11862,1995,fatherofthebridepartii
5,6,Heat,Action Crime Thriller,113277,949,1995,heat
6,7,Sabrina,Comedy Romance,114319,11860,1995,sabrina
7,8,Tom and Huck,Adventure Children,112302,45325,1995,tomandhuck
8,9,Sudden Death,Action,114576,9091,1995,suddendeath
9,10,GoldenEye,Action Adventure Thriller,113189,710,1995,goldeneye


In [32]:
# import pickle


In [33]:
# pickle.dump(movies.to_dict(),open("movies2.pkl","wb"))


In [34]:
# pickle.dump(similar_movies2,open("similar.pkl","wb"))


In [35]:
# similar_movies2=similar_movies[:10000]


In [36]:
# similar_movies3=similar_movies[10000:]
# pickle.dump(similar_movies3,open("similar2.pkl","wb"))


In [37]:
movies.to_dict()


{'movieId': {0: 1,
  1: 2,
  2: 3,
  3: 4,
  4: 5,
  5: 6,
  6: 7,
  7: 8,
  8: 9,
  9: 10,
  10: 11,
  11: 12,
  12: 13,
  13: 14,
  14: 15,
  15: 16,
  16: 17,
  17: 18,
  18: 19,
  19: 20,
  20: 21,
  21: 22,
  22: 23,
  23: 24,
  24: 25,
  25: 26,
  26: 27,
  27: 28,
  28: 29,
  29: 30,
  30: 31,
  31: 32,
  32: 33,
  33: 34,
  34: 35,
  35: 36,
  36: 37,
  37: 38,
  38: 39,
  39: 40,
  40: 41,
  41: 42,
  42: 43,
  43: 44,
  44: 45,
  45: 46,
  46: 47,
  47: 48,
  48: 49,
  49: 50,
  50: 51,
  51: 52,
  52: 53,
  53: 54,
  54: 55,
  55: 56,
  56: 57,
  57: 58,
  58: 59,
  59: 60,
  60: 61,
  61: 62,
  62: 63,
  63: 64,
  64: 65,
  65: 66,
  66: 67,
  67: 68,
  68: 69,
  69: 70,
  70: 71,
  71: 72,
  72: 73,
  73: 74,
  74: 75,
  75: 76,
  76: 77,
  77: 78,
  78: 79,
  79: 80,
  80: 81,
  81: 82,
  82: 83,
  83: 84,
  84: 85,
  85: 86,
  86: 87,
  87: 88,
  88: 89,
  89: 90,
  90: 92,
  91: 93,
  92: 94,
  93: 95,
  94: 96,
  95: 97,
  96: 98,
  97: 99,
  98: 100,
  99: 101,
  100:

In [38]:
def listgenre(value):
    value = value.lower().split()
    return value


movies['genres'] = movies['genres'].apply(listgenre)


In [39]:
movies90s = movies.loc[(movies['year'] > 1990) & (movies['year'] < 2000)]


In [40]:
movies80s = movies.loc[(movies['year'] > 1980) & (movies['year'] < 1990)]


In [41]:
movies2000s = movies.loc[(movies['year'] > 2000) & (movies['year'] < 2010)]


In [42]:
latest = movies.loc[(movies['year'] > 2010) & (movies['year'] < 2020)]


In [43]:
import pickle


In [44]:
pickle.dump(movies90s.to_dict(), open("movies90s.pkl", "wb"))


In [45]:
pickle.dump(movies80s.to_dict(), open("movies80s.pkl", "wb"))


In [46]:
pickle.dump(movies2000s.to_dict(), open("movies2000s.pkl", "wb"))


In [47]:
pickle.dump(latest.to_dict(), open("latest.pkl", "wb"))


In [48]:
import pickle
import pandas as pd

a90s = pickle.load(open("movies90s.pkl", "rb"))
a80s = pickle.load(open("movies80s.pkl", "rb"))
a2000s = pickle.load(open("movies2000s.pkl", "rb"))
alatest = pickle.load(open("latest.pkl", "rb"))

a90s = pd.DataFrame(a90s)
a80s = pd.DataFrame(a80s)
a80s = a80s.reset_index()

a2000s = pd.DataFrame(a2000s)
alatest = pd.DataFrame(alatest)

a80s


,index,movieId,title,genres,imdbId,tmdbId,year,oldtitle
0,140,142,Shadows (Cienie),[drama],94878,525153,1988,shadows(cienie)
1,536,541,Blade Runner,"[action, sci-fi, thriller]",83658,78,1982,bladerunner
2,584,592,Batman,"[action, crime, thriller]",96895,268,1989,batman
3,602,610,Heavy Metal,"[action, adventure, animation, horror, sci-fi]",82509,11827,1981,heavymetal
4,668,681,Coup de torchon (Clean Slate),[crime],82206,35797,1981,coupdetorchon(cleanslate)
...,...,...,...,...,...,...,...,...
2210,24851,122451,Dark Tower,"[horror, thriller]",92831,106280,1989,darktower
2211,24896,122553,Assault of the Party Nerds,[comedy],96839,30027,1989,assaultofthepartynerds
2212,24930,122623,Oliver Twist,"[crime, drama]",84438,49483,1982,olivertwist
2213,24940,122647,Movers & Shakers,[comedy],89628,89743,1985,movers&shakers


In [49]:
from numpy import random


def a80ss():
    idxlist = []
    suggest = []
    suggest_poster = []
    released_date = []
    overview = []
    for num in range(0, 20):
        idxlist.append(random.randint(2100))
    for j in idxlist:
        index_id = a80s.loc[j, 'tmdbId']
        print(j, index_id)
        suggest.append(a80s.loc[j, 'title'])
    return suggest


In [50]:
a80ss()


1402 35151
1170 20980
976 14774
1949 18741
1481 15301
1827 61755
1585 42011
29 572997
1302 64945
10 253632
840 14372
1910 31400
312 26789
1191 27609
408 34760
435 19259
1031 10863
155 793
2089 32079
1109 13945


['Hello Again',
 'Liquid Sky',
 "Cheech & Chong's The Corsican Brothers",
 'Fearless Hyena II (Long teng hu yue)',
 'Twilight Zone: The Movie',
 '1990: The Bronx Warriors (1990: I guerrieri del Bronx)',
 'Taxing Woman, A (Marusa no onna)',
 'Invitation, The (Zaproszenie)',
 'That Was Then... This Is Now',
 'Baton Rouge (Bâton rouge)',
 'Leviathan',
 'Balkan Spy (Balkanski spijun)',
 'No Mercy',
 'Personal Best',
 "'night Mother",
 'Volunteers',
 'Eye of the Needle',
 'Blue Velvet',
 'Cohen and Tate',
 'Sword and the Sorcerer, The']

In [51]:
a80s.isnull().sum()


index       0
movieId     0
title       0
genres      0
imdbId      0
tmdbId      0
year        0
oldtitle    0
dtype: int64